In [ ]:
# Import libraries and set default options
import datetime
import calendar
import random
import math
import time
import pandas as pd
import numpy as np
from array import array
import pickle

import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import NMF, PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import cross_validate

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import  datasets, layers, models
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, SimpleRNN, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
cifar100_labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
def returnBlockOneAndTwoTrainSets():
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data(label_mode='fine')

    class_names = test_labels.reshape(-1) 
    # Normalise pixel values 0-1
    train_images_normalised, test_images_normalised = train_images / 255.0, test_images / 255.0
    
    # Create a list of all classes
    classes = np.arange(100)
    
    # Randomly shuffle the classes
    np.random.shuffle(classes)
    
    block1_classes = classes[:50]
    block2_classes = classes[50:]

    train_labels = train_labels.reshape(-1)
    test_labels = test_labels.reshape(-1)

    train_mask1 = np.isin(train_labels, block1_classes)
    train_mask2 = np.isin(train_labels, block2_classes)
    test_mask1 = np.isin(test_labels, block1_classes)
    test_mask2 = np.isin(test_labels, block2_classes)
    
    # Split the data into two subsets
    # train_images_1, train_labels_1 = train_images_normalised[train_mask1], train_labels[train_mask1]
    # train_images_2, train_labels_2 = train_images_normalised[train_mask2], train_labels[train_mask2]
    # test_images_1, test_labels_1 = test_images_normalised[test_mask1], test_labels[test_mask1]
    # test_images_2, test_labels_2 = test_images_normalised[test_mask2], test_labels[test_mask2]
    train_images_1 = np.take(train_images_normalised, np.where(train_mask1)[0], axis=0)
    train_labels_1 = np.take(train_labels, np.where(train_mask1)[0])
    train_images_2 = np.take(train_images_normalised, np.where(train_mask2)[0], axis=0)
    train_labels_2 = np.take(train_labels, np.where(train_mask2)[0])
    test_images_1 = np.take(test_images_normalised, np.where(test_mask1)[0], axis=0)
    test_labels_1 = np.take(test_labels, np.where(test_mask1)[0])
    test_images_2 = np.take(test_images_normalised, np.where(test_mask2)[0], axis=0)
    test_labels_2 = np.take(test_labels, np.where(test_mask2)[0])

    return block1_classes, train_images_1, train_labels_1, test_images_1, test_labels_1, block2_classes, train_images_2, train_labels_2, test_images_2, test_labels_2




In [ ]:
def evaluateAndAnalyse(model, history, test_images, test_labels, block_classes):
    subset_class_names = np.unique([cifar100_labels[i] for i in block_classes])

    # Generate predictions (using subset 1 for example)
    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)
    
    true_labels_text = np.array([cifar100_labels[label] for label in test_labels])
    predicted_labels_text = np.array([cifar100_labels[label] for label in predicted_labels])
    
    # Confusion matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(true_labels_text, predicted_labels_text, labels=subset_class_names) 
    
    
    # Plotting the confusion matrix
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    # Add labels to the plot
    tick_marks = np.arange(len(subset_class_names))
    plt.xticks(tick_marks, subset_class_names, rotation=45, ha='right')
    plt.yticks(tick_marks, subset_class_names)
    
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    # from sklearn.metrics import classification_report
    print(classification_report(test_labels, np.argmax(predictions, axis=1)))

    report = classification_report(true_labels_text, predicted_labels_text, labels=subset_class_names, output_dict=True)

    # Convert report to DataFrame
    df = pd.DataFrame(report).transpose()
    
    # Sort by F1-score
    df = df.sort_values(by='f1-score', ascending=False)
    
    # Print the DataFrame
    print(df.to_markdown(numalign="left", stralign="left"))


    overall_accuracy = report['accuracy'] 
    print(f"Overall accuracy: {overall_accuracy:.4f}") 

In [ ]:
def plotTrainingAndValidationLoss(train_loss, val_loss ):
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    # modelConfig["plots"].append(plt)
    plt.show()
    

def plotTrainingAndValidationAccuracy(train_acc, val_acc ):
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    # modelConfig["plots"].append(plt)
    plt.show()
    
def plotModel(history):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy'] 
    val_acc = history.history['val_accuracy']

    plotTrainingAndValidationLoss(train_loss, val_loss)
    plotTrainingAndValidationAccuracy(train_acc, val_acc)

In [ ]:
def compileAndTrainModel1(train_images, train_labels, test_images, test_labels):
    # Standard basic model
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    # Flatten the output from convolutional layers
    model.add(layers.Flatten())
    
    # Hidden Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels, epochs=10, 
                        validation_data=(test_images, test_labels))

    return model, history

In [ ]:
def compileAndTrainModel2(train_images, train_labels, test_images, test_labels):
    # Standard model but with data augmentation through image generation
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    # Flatten the output from convolutional layers
    model.add(layers.Flatten())
    
    # Hidden Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_generator = datagen.flow(train_images, train_labels, batch_size=32) 
    history = model.fit(train_generator, epochs=10, validation_data=(test_images, test_labels))
    
    # Train the model
    # history = model.fit(train_images, train_labels, epochs=10, 
    #                     validation_data=(test_images, test_labels))

    return model, history

In [ ]:
def compileAndTrainModel3(train_images, train_labels, test_images, test_labels):
    # Standard basic model 20 epochs
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    # Flatten the output from convolutional layers
    model.add(layers.Flatten())
    
    # Hidden Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels, epochs=20, 
                        validation_data=(test_images, test_labels))

    return model, history

In [ ]:
def compileAndTrainModel4(train_images, train_labels, test_images, test_labels):
    # Standard basic model ELU
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    # Flatten the output from convolutional layers
    model.add(layers.Flatten())
    
    # Hidden Dense layers
    model.add(layers.Dense(128, activation='elu'))
    model.add(layers.Dense(128, activation='elu'))
    model.add(layers.Dense(128, activation='elu'))
    model.add(layers.Dense(128, activation='elu'))
    model.add(layers.Dense(128, activation='elu'))
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels, epochs=10, 
                        validation_data=(test_images, test_labels))

    return model, history

In [ ]:
def compileAndTrainModel5(train_images, train_labels, test_images, test_labels):
    # Standard basic model Leaky RELU
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    # Flatten the output from convolutional layers
    model.add(layers.Flatten())
    
    # Hidden Dense layers
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3))  # LeakyReLU with alpha=0.3
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3))  # LeakyReLU with alpha=0.3
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3))  # LeakyReLU with alpha=0.3
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3))  # LeakyReLU with alpha=0.3
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3))  # LeakyReLU with alpha=0.3
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels, epochs=10, 
                        validation_data=(test_images, test_labels))

    return model, history

In [ ]:
def compileAndTrainModel6(train_images, train_labels, test_images, test_labels):
    # Standard basic model logistic / sigmoid
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

    train_labels_encoded = tf.keras.utils.to_categorical(train_labels, num_classes=100)
    test_labels_encoded = tf.keras.utils.to_categorical(test_images, num_classes=100)
    
    # Train the model
    history = model.fit(train_images, train_labels_encoded, epochs=10, 
                        validation_data=(test_images, test_labels_encoded))

    return model, history

In [ ]:
def compileAndTrainModel7(train_images, train_labels, test_images, test_labels):
    # Standard basic model skip connections
    model = models.Sequential()

    # Define the input tensor
    inputs = Input(shape=(32, 32, 3))
    
    # Convolutional layers with skip connections
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x_skip = x  # Store the output for the skip connection
    
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)

    x_skip = layers.Conv2D(128, (1, 1), strides=4, activation='relu')(x_skip)  # Adjust channels and downsample
    x = layers.add([x, x_skip]) 
    
    # Flatten and dense layers with ReLU
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(100, activation='softmax')(x)
    

    model = models.Model(inputs=inputs, outputs=outputs)
    
    # Compile the model
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels, epochs=10, 
                        validation_data=(test_images, test_labels))

    return model, history

In [ ]:
def compileAndTrainModel8(train_images, train_labels, test_images, test_labels):
    # Standard basic model Leaky RELU, 30 epochs, early stopping
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    
    # Flatten the output from convolutional layers
    model.add(layers.Flatten())
    
    # Hidden Dense layers
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3)) 
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3)) 
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3)) 
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3)) 
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU(alpha=0.3)) 
    
    # Output layer with 100 units for 100 classes
    model.add(layers.Dense(100, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels, epochs=30, 
                        validation_data=(test_images, test_labels),
                        callbacks=[early_stopping])

    return model, history

In [ ]:
block1_classes, train_images_1, train_labels_1, test_images_1, test_labels_1, block2_classes, train_images_2, train_labels_2, test_images_2, test_labels_2 = returnBlockOneAndTwoTrainSets()


In [ ]:
model, history = compileAndTrainModel1(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model, history, test_images_1, test_labels_1, block1_classes)
plotModel(history)

In [ ]:
model2, history2 = compileAndTrainModel2(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model2, history2, test_images_1, test_labels_1, block1_classes)
plotModel(history2)

In [ ]:
model3, history3 = compileAndTrainModel3(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model3, history3, test_images_1, test_labels_1, block1_classes)
plotModel(history3)

In [ ]:
model4, history4 = compileAndTrainModel4(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model4, history4, test_images_1, test_labels_1, block1_classes)
plotModel(history4)

In [ ]:
model5, history5 = compileAndTrainModel(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model5, history5, test_images_1, test_labels_1, block1_classes)
plotModel(history5)

In [ ]:
# fails due to dimension mismmatches
# model6, history6 = compileAndTrainModel6(train_images_1, train_labels_1, test_images_1, test_labels_1)
# evaluateAndAnalyse(model6, history6, test_images_1, test_labels_1, block1_classes)
# plotModel(history6)

In [ ]:
model7, history7 = compileAndTrainModel7(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model7, history7, test_images_1, test_labels_1, block1_classes)
plotModel(history7)

In [ ]:
model8, history8 = compileAndTrainModel8(train_images_1, train_labels_1, test_images_1, test_labels_1)
evaluateAndAnalyse(model8, history8, test_images_1, test_labels_1, block1_classes)
plotModel(history8)

In [ ]:
kerasFileName = 'part2saves/part2imgclass_bestmodel.h5'
model8.save(kerasFileName)

In [ ]:
kerasFileName = 'part2saves/part2imgclass_besthistory.h5'
history8.save(kerasFileName)

In [ ]:
# modelFileName = 'part2saves/part2imgclass_bestmodel.pkl'
# modelFile = open(modelFileName, 'ab')
# pickle.dump(model8, modelFile)                    
# modelFile.close()

In [ ]:
modelFileName = 'part2saves/part2imgclass_besthistory.pkl'
modelFile = open(modelFileName, 'ab')
pickle.dump(history8.history, modelFile)                    
modelFile.close()

In [ ]:
type(history8)

In [ ]:
best_model = model8
best_history = history8


## Autoencoder Modelling: For Block 1 Images 


In [ ]:
def evaluateAutoencoder(autoencoder, test_images):
    mse = autoencoder.evaluate(test_images, test_images)  # Calculate MSE on the test set
    print("Mean Squared Error:", mse)

In [ ]:
def visualTestAutoencoder(autoencoder, test_images):
    reconstructed_images = autoencoder.predict(test_images)

    for i in range(5):
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1)
        plt.imshow(test_images[i])
        plt.title("Original")
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(reconstructed_images[i])
        plt.title("Reconstructed")
        plt.axis('off')
        plt.show()

In [ ]:
def plotAutoencoder(history):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    # train_acc = history.history['accuracy'] 
    # val_acc = history.history['val_accuracy']

    plotTrainingAndValidationLoss(train_loss, val_loss)
    # plotTrainingAndValidationAccuracy(train_acc, val_acc)

In [ ]:
def compileAndTrainAutoencoder1(original_model,train_images, train_labels, test_images, test_labels):
    encoder = models.Sequential()
    for layer in original_model.layers[:5]:  # Take the first 5 layers (convolutions and flatten)
        encoder.add(layer)
    
    # --- Decoder ---
    decoder = models.Sequential()
    decoder.add(layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same', input_shape=(4, 4, 128)))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same'))  # Output layer

    # --- Autoencoder ---
    autoencoder = models.Sequential([encoder, decoder])
    
    # Compile the model
    autoencoder.compile(optimizer='adam', loss='mse')
    
    # Train the autoencoder
    history = autoencoder.fit(train_images, train_images, epochs=10, batch_size=32, validation_split=0.1)
    
    return autoencoder, history

In [ ]:
autoencoder, autoencoder_history = compileAndTrainAutoencoder1(best_model,train_images_1, train_labels_1, test_images_1, test_labels_1)
plotAutoencoder(autoencoder_history)
visualTestAutoencoder(autoencoder, test_images_1)
evaluateAutoencoder(autoencoder, test_images_1)

In [ ]:
kerasFileName = 'part2saves/part2imgclass_autoencoder.h5'
autoencoder.save(kerasFileName)

modelFileName = 'part2saves/part2imgclass_autoencoderhistory.pkl'
modelFile = open(modelFileName, 'ab')
pickle.dump(autoencoder_history, modelFile)                    
modelFile.close()

In [ ]:
# modelFileName = 'part2saves/part2imgclass_autoencoder.pkl'
# modelFile = open(modelFileName, 'ab')
# pickle.dump(autoencoder, modelFile)                    
# modelFile.close()

## Transfer Learning for Block 2 Images

In [ ]:
def compileAndTrainTransferModel(model, train_images, train_labels, test_images, test_labels):
    # Freeze the convolutional layers
    for layer in model.layers[:5]:  # The first 5 layers are convolutional
        layer.trainable = False
    
    # Create a new model for transfer learning
    transfer_model = models.Sequential()
    transfer_model.add(model)  # Add the original model as the base
    
    # Add new classification layers
    transfer_model.add(layers.Dense(128))
    transfer_model.add(layers.LeakyReLU(alpha=0.3)) 
    transfer_model.add(layers.Dense(128))
    transfer_model.add(layers.LeakyReLU(alpha=0.3)) 
    transfer_model.add(layers.Dense(128))
    transfer_model.add(layers.LeakyReLU(alpha=0.3)) 
    # transfer_model.add(layers.Dense(128))
    # transfer_model.add(layers.LeakyReLU(alpha=0.3)) 
    # transfer_model.add(layers.Dense(128))
    # transfer_model.add(layers.LeakyReLU(alpha=0.3)) 

    
    transfer_model.add(layers.Dense(50, activation='softmax'))  # 50 output classes
    
    # Compile the transfer learning model
    transfer_model.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])
    
    # Train the transfer learning model on the second subset
    history = transfer_model.fit(train_images, train_labels, epochs=30, 
                                validation_data=(test_images, test_labels),
                                callbacks=[early_stopping])

    return transfer_model, history

In [ ]:
def fineTuneTransferModel(model, transfer_model, train_images, train_labels, test_images, test_labels):
    for layer in model.layers[3:]:  
        layer.trainable = True
    
    transfer_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

    history = transfer_model.fit(train_images, train_labels, epochs=10, 
                                validation_data=(test_images, test_labels))

    return transfer_model, history

In [ ]:
modelBlock2FromScratch, historyBlock2FromScratch = compileAndTrainModel8(train_images_2, train_labels_2, test_images_2, test_labels_2)
evaluateAndAnalyse(modelBlock2FromScratch, historyBlock2FromScratch, test_images_2, test_labels_2, block1_classes)
plotModel(historyBlock2FromScratch)

In [ ]:
modelBlock2FromTransfer, historyBlock2FromTransfer = compileAndTrainTransferModel(best_model, train_images_2, train_labels_2, test_images_2, test_labels_2)
evaluateAndAnalyse(modelBlock2FromTransfer, historyBlock2FromTransfer, test_images_2, test_labels_2, block2_classes)
plotModel(historyBlock2FromTransfer)

In [ ]:
modelBlock2FromFineTune, historyBlock2FromFineTune = fineTuneTransferModel(best_model, modelBlock2FromTransfer, train_images_2, train_labels_2, test_images_2, test_labels_2)
evaluateAndAnalyse(modelBlock2FromFineTune, historyBlock2FromFineTune, test_images_2, test_labels_2, block2_classes)
plotModel(historyBlock2FromFineTune)

In [ ]:
# test_images_2
modelFileName = 'part2saves/part2_block2_classes.pkl'
modelFile = open(modelFileName, 'ab')
pickle.dump(block2_classes, modelFile)                    
modelFile.close()

In [ ]:
block2_classes